<a href="https://colab.research.google.com/github/benjaminnigjeh/keyProteoforms/blob/main/dimensionalityReduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and dependencies

Dimensionality reduction is performed by using tensorfolw and keras as frontend and backend, respectively. The datasets are prepared by using numpy and pandas libraries.

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import TensorBoard
from google.colab import files

# Set up the tensorboard callback function

In [2]:
NAME = "Unsupervised_clustering"
tensorboard = TensorBoard(
    log_dir='/content/{}'.format(NAME))

# Upload dataset from a local folder


In [28]:
n_dim = 37
dataset = files.upload_file('dataset')
df = pd.read_csv('/content/dataset')
X = df.copy()
Y = X.pop("target")
X_np = np.array(X)
x_train = np.reshape(X_np,(-1, n_dim, n_dim, 1))
y_train = np.array(Y)


Saved ms1diseaseinsoluble.csv to /content/dataset


# Encoder and decoder architure and VAE class

In [29]:
latent_dim = 2


class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }



#Training function

In [30]:
def vae_training(random_seed):
  np.random.seed(random_seed)
  tf.random.set_seed(random_seed)
  encoder_inputs = keras.Input(shape=(n_dim, n_dim, 1))
  x = layers.Flatten()(encoder_inputs)
  x = layers.Dense(120 , activation="relu")(x)
  x = layers.Dense(120 , activation="relu")(x)
  x = layers.Dense(16, activation="relu")(x)
  z_mean = layers.Dense(latent_dim, name="z_mean")(x)
  z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
  z = Sampling()([z_mean, z_log_var])
  encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
  latent_inputs = keras.Input(shape=(latent_dim,))
  x = layers.Dense(n_dim * n_dim * 1, activation="relu")(latent_inputs)
  x = layers.Dense(n_dim * n_dim * 1, activation="sigmoid")(x)
  decoder_outputs = layers.Reshape((n_dim, n_dim, 1))(x)
  decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
  vae = VAE(encoder, decoder)
  vae.compile(optimizer=keras.optimizers.Adam())
  history = vae.fit(x_train, epochs=50, callbacks=[tensorboard])
  loss = history.history['loss']
  return loss

#Training with multiple random initiations

In [ ]:
pellet_loss = []

for i in [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]:
  pellet_loss.append(vae_training(i))

soluble_loss = []

for i in [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]:
  soluble_loss.append(vae_training(i))

#plotting loss versus epoch

In [ ]:
data_array = np.array(soluble_loss)
mean_values = np.mean(data_array, axis=0)
std_values = np.std(data_array, axis=0)
x = np.arange(1, data_array.shape[1] + 1)

plt.figure(figsize=(8, 5))
plt.errorbar(x, mean_values, yerr=std_values, fmt='o', color='blue', label='soluble loss', capsize=5)

data_array = np.array(pellet_loss)
mean_values = np.mean(data_array, axis=0)
std_values = np.std(data_array, axis=0)
x = np.arange(1, data_array.shape[1] + 1)

plt.errorbar(x, mean_values, yerr=std_values, fmt='o', color='red', label='pellet loss', capsize=5)


# Adding labels and title
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim(600, 1200)
plt.title('')
plt.legend()

# Display the plot
plt.grid(True)
plt.show()


#visualizing the latent layer

In [ ]:
def plot_label_clusters(vae, data, labels):
    z_mean, _, _ = vae.encoder.predict(data)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=labels)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.show()

plot_label_clusters(vae, x_train, y_train)